In [1]:
import plotly.express as px
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('flights').getOrCreate()

23/01/03 13:56:12 WARN Utils: Your hostname, MacBook-Air-di-Teodoro.local resolves to a loopback address: 127.0.0.1; using 192.168.135.184 instead (on interface en0)
23/01/03 13:56:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/03 13:56:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# read the schema from the json file
import json


with open("../../util/schema.json","r") as f:
    schema = StructType.fromJson(json.load(f))

In [4]:

df_cleaned = spark.read.csv('../../data.nosync/cleaned/cleaned_flights.csv/',schema=schema,header=True)

In [5]:
num_airports = df_cleaned.select("Origin").union(df_cleaned.select("Dest")).distinct().count()

In [6]:
num_airports

316

In [6]:
print(df_cleaned.count())
print(len(df_cleaned.columns))

6246739
43


In [3]:
df = spark.read.csv('../../data.nosync/', inferSchema=True, header=True)

In [15]:
import pandas as pd
features = pd.read_csv("../preprocessing/features.csv")["Features"].tolist()
features

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'Origin',
 'Dest',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Distance',
 'DistanceGroup']

In [16]:
df_cleaned = df.select(features)

In [17]:
# get the number of rows that contains in any column a null value
before = df_cleaned.count()
after = df_cleaned.na.drop().count()


In [19]:
total = before + after
null = before-after

In [22]:
# plot total and null as pie chart
fig = px.pie(names=["null", "total"], values=[null, total], title="Null values")
fig.show()

In [6]:
normal = df.filter(df['Diverted']==0).count()
diverted = df.filter(df['Diverted']==1).count()

In [9]:
# plot normal and diverted flights as pie chart with percentages
fig = px.pie(names=['Normal', 'Diverted'], values=[normal, diverted], title='Normal and diverted flights')
fig.show()

In [10]:
normal

6355322

In [11]:
diverted

14160

In [6]:
import pandas as pd
import plotly.express as px

In [11]:
cd_df = pd.read_csv("../../util/cancelled_diverted.csv")
# sort by date
cd_df = cd_df.sort_values(by=['FlightDate'])

In [12]:
cd_df.head()

,FlightDate,Diverted,Cancelled
151,2013-01-01,30.0,116.0
152,2013-01-02,27.0,145.0
146,2013-01-03,19.0,123.0
135,2013-01-04,19.0,136.0
139,2013-01-05,18.0,52.0


In [15]:
# add month and day of week columns
cd_df['Month'] = pd.DatetimeIndex(cd_df['FlightDate']).month
cd_df['DayOfWeek'] = pd.DatetimeIndex(cd_df['FlightDate']).dayofweek


In [28]:
# save the dataframe to a csv file
cd_df.to_csv("../../util/cancelled_diverted.csv", index=False)

In [16]:
# group by month, make the avg of cancelled and diverted and sort by month
cd_df_month = cd_df.groupby(['Month']).mean().reset_index().sort_values(by=['Month'])

/var/folders/tx/5_6_zg1d0fd4zjwzxg1ztrth0000gn/T/ipykernel_5640/1497372947.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [25]:
cd_df_week = cd_df.groupby(['DayOfWeek']).mean().reset_index().sort_values(by=['DayOfWeek'])

/var/folders/tx/5_6_zg1d0fd4zjwzxg1ztrth0000gn/T/ipykernel_5640/1737658621.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [21]:
fig = px.line(cd_df, x="FlightDate", y=["Cancelled"], title="Cancelled and diverted flights per day")
fig.show()

In [23]:
fig = px.line(cd_df, x="FlightDate", y=["Diverted"], title="Cancelled and diverted flights per day")
fig.show()

In [24]:
# plot the number of cancelled and diverted flights per day
fig = px.line(cd_df_month, x="Month", y=["Cancelled"], title="Cancelled and diverted flights per day")
fig.show()

In [20]:
fig = px.line(cd_df_month, x="Month", y=["Diverted"], title="Cancelled and diverted flights per day")
fig.show()

In [26]:
fig = px.line(cd_df_week, x="DayOfWeek", y=["Cancelled"], title="Cancelled and diverted flights per day")
fig.show()

In [27]:
fig = px.line(cd_df_week, x="DayOfWeek", y=["Diverted"], title="Cancelled and diverted flights per day")
fig.show()